In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from routingpy.routers import MapboxOSRM
import numpy as np

mb = MapboxOSRM(api_key = "pk.eyJ1IjoibXVoYW1tYWRyaWZxeSIsImEiOiJjbGptenU2Z2QwdzJjM2xyN2Rham1ndXhoIn0.5_qenbJijy4l7c8xhJBkkQ")

In [2]:
def mb_isochrone(gdf, time = [3, 5, 7, 10, 13, 15, 18, 20], profile = "driving"):

    # Grab X and Y values in 4326
    gdf['LON_VALUE'] = gdf.to_crs(4326).geometry.x
    gdf['LAT_VALUE'] = gdf.to_crs(4326).geometry.y

    coordinates = gdf[['LON_VALUE', 'LAT_VALUE']].values.tolist()

    # Build a list of shapes
    isochrone_shapes = []

    if type(time) is not list:
        time = [time]

    # Use minutes as input, but the API requires seconds
    time_seconds = [60 * x for x in time]

    # Given the way that routingpy works, we need to iterate through the list of 
    # coordinate pairs, then iterate through the object returned and extract the 
    # isochrone geometries.  
    for c in coordinates:
        iso_request = mb.isochrones(locations = c, profile = profile,
                                    intervals = time_seconds, polygons = "true")

        for i in iso_request:
            iso_geom = Polygon(i.geometry[0])
            isochrone_shapes.append(iso_geom)

    # Here, we re-build the dataset but with isochrone geometries
    df_values = gdf.drop(columns = ['geometry', 'LON_VALUE', 'LAT_VALUE'])

    time_col = time * len(df_values)

    # We'll need to repeat the dataframe to account for multiple time intervals
    df_values_rep = pd.DataFrame(np.repeat(df_values.values, len(time_seconds), axis = 0))
    df_values_rep.columns = df_values.columns

    isochrone_gdf = gpd.GeoDataFrame(
        data = df_values_rep,
        geometry = isochrone_shapes,
        crs = 4326
    )

    isochrone_gdf['time'] = time_col

    # We are sorting the dataframe in descending order of time to improve visualization
    # (the smallest isochrones should go on top, which means they are plotted last)
    isochrone_gdf = isochrone_gdf.sort_values('time', ascending = False)

    return(isochrone_gdf)

In [85]:
stasiun = gpd.read_file("terminal.shp")
stasiun.explore()

In [5]:
stasiun

,OBJECTID,NAMOBJ,FCODE,REMARK,METADATA,SRS_ID,JLBTRM,JLDTRM,KATTRM,KODTRM,KONKON,MGTTRM,PARTRM,STATRM,THBTRM,THTTRM,TIPTRM,THNOPR,geometry
0,1,Terminal Pulogadung,CA01080200,Terminal Penumpang Tipe B,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,2,NaN,POINT (106.90933 -6.18234)
1,2,Terminal Pulo Gebang,CA01080200,Terminal Penumpang Tipe A,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,1,NaN,POINT (106.95217 -6.21199)
2,3,Terminal Kampung Rambutan,CA01080200,Terminal Penumpang Tipe A,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,1,NaN,POINT (106.88232 -6.30945)
3,4,Terminal Kalideres,CA01080200,Terminal Penumpang Tipe A,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,1,NaN,POINT (106.70547 -6.15473)
4,5,Terminal Grogol,CA01080200,Terminal Penumpang Tipe B,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,2,NaN,POINT (106.79075 -6.16651)
5,6,Terminal Tanjung Priok,CA01080200,Terminal Penumpang Tipe A,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,1,NaN,POINT (106.88110 -6.10965)
6,7,Terminal Senen,CA01080200,Terminal Penumpang Tipe B,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,2,NaN,POINT (106.84232 -6.17382)
7,8,Terminal Kampung Melayu,CA01080200,Terminal Penumpang Tipe B,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,2,NaN,POINT (106.86669 -6.22478)
8,9,Terminal Blok M,CA01080200,Terminal Penumpang Tipe B,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,2,NaN,POINT (106.80090 -6.24346)
9,10,Terminal Pasar Minggu,CA01080200,Terminal Penumpang Tipe B,NaN,NaN,0.0,0.0,0,NaN,0,NaN,0,2,NaN,NaN,2,NaN,POINT (106.84320 -6.28357)


In [68]:
ragunan = gpd.read_file("stasiun/Ragunan.shp")

In [69]:
ragunan_iso = mb_isochrone(ragunan, time = [5, 10, 15, 20], profile="driving-traffic")
ragunan_iso.explore(column="time",  # make choropleth based on "time" column
    tooltip="time",  # show "time" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB positron",  # use "CartoDB positron" tiles
    cmap="Blues",  # use "Blues" matplotlib colormap
    style_kwds=dict(color="black"),)  # use black outline)

In [15]:
import folium
from folium import plugins

In [70]:
ragunan_iso_cycle = mb_isochrone(ragunan, time = [5, 10, 15, 20], profile="cycling")
ragunan_iso_cycle.explore(column="time",  # make choropleth based on "time" column
    tooltip="time",  # show "time" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB positron",  # use "CartoDB positron" tiles
    cmap="Blues",  # use "Blues" matplotlib colormap
    style_kwds=dict(color="black"),)  # use black outline)

In [71]:
ragunan_iso_walking = mb_isochrone(ragunan, time = [5, 10, 15, 20], profile="walking")
ragunan_iso_walking.explore(column="time",  # make choropleth based on "time" column
    tooltip="time",  # show "time" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB positron",  # use "CartoDB positron" tiles
    cmap="Blues",  # use "Blues" matplotlib colormap
    style_kwds=dict(color="black"),)  # use black outline)

In [72]:
ragunan_map = ragunan_iso.explore(column="time",  # make choropleth based on "time" column
    tooltip="time",  # show "time" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    scheme="naturalbreaks",
    legend=True,
    cmap="Blues",  # use "Blues" matplotlib colormap
    legend_kwds=dict(colorbar=False),
    style_kwds=dict(color="black"),  # use black outline
    name="Berkendara Motor",)

ragunan_iso_cycle.explore(
    m=ragunan_map,
    column="time",  # make choropleth based on "time" column
    tooltip="time",  # show "time" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    scheme="naturalbreaks",
    legend=True,
    cmap="Greens",  # use "Greens" matplotlib colormap
    legend_kwds=dict(colorbar=False),
    style_kwds=dict(color="black"),  # use black outline
    name="Bersepeda",)

ragunan_iso_walking.explore(
    m=ragunan_map,
    column="time",  # make choropleth based on "time" column
    tooltip="time",  # show "time" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    scheme="naturalbreaks",
    legend=True,
    cmap="Oranges",  # use "Oranges" matplotlib colormap
    legend_kwds=dict(colorbar=False),
    style_kwds=dict(color="black"),  # use black outline
    name="Berjalan Kaki",)

stasiun.explore(
    m=ragunan_map,  # pass the map object
    color="red",  # use red color on all points
    marker_kwds=dict(radius=5, fill=True),  # make marker radius 10px with fill
    tooltip=["NAMOBJ", "REMARK"],  # show "NAMOBJ" & "REMARK" column in the tooltip
    tooltip_kwds=dict(labels=True),  # do not show column label in the tooltip
    name="Stasiun",  # name of the layer in the map
)

plugins.Geocoder().add_to(ragunan_map)

folium.LayerControl().add_to(ragunan_map) # use folium to add layer control

title_html = '''
            <link href="https://fonts.googleapis.com/css?family=Outfit:300,300i,400,400i,600,600i,700,700i|Raleway:300,300i,400,400i,500,500i,600,600i,700,700i|Poppins:300,300i,400,400i,500,500i,600,600i,700,700i | Montserrat: 300,300i,400,400i,500,500i,600,600i,700,700i" rel="stylesheet">
             <style>
    #header {
        border-radius: opx;
            margin: auto;
            width: 100%;
            position: static;
            z-index: 5;
            font: bold 30px 'Outfit';
            text-align: center;
            color: black;
            background-image: url('https://mamedrifqy.github.io/assets/img/background.png');
            background-blend-mode: overlay;
    }
            </style>
            
            <header id="header">Peta <i>Isochrone</i> Terminal Ragunan</header>

            
        
            
             '''
ragunan_map.get_root().html.add_child(folium.Element(title_html))

ragunan_map  # show map

C:\Users\ACER\anaconda3\lib\site-packages\mapclassify\classifiers.py:1775: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 4.
  warnings.warn(
C:\Users\ACER\anaconda3\lib\site-packages\mapclassify\classifiers.py:1775: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 4.
  warnings.warn(
C:\Users\ACER\anaconda3\lib\site-packages\mapclassify\classifiers.py:1775: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 4.
  warnings.warn(


In [84]:
ragunan_map.save("ragunan_map.html")

In [82]:
stasiun.explore(
    color="red",  # use red color on all points
    marker_kwds=dict(radius=5, fill=True),  # make marker radius 10px with fill
    tooltip=["NAMOBJ", "REMARK"],  # show "name" column in the tooltip
    tooltip_kwds=dict(labels=True),  # do not show column label in the tooltip
    name="Stasiun",  # name of the layer in the map
)